In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wave as w
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)
w.Spectrum.k = np.linspace(2 * np.pi / (1.0 * w.MICRO), 2 * np.pi / (300.0 * w.NANO), 500)

# Example usage of simulated materials
This notebook is ment as an reference on how to use `Spectrum` and `Material` classes,
and for visual test of the implementation.

### Generate an example spectrum

In [ ]:
s = w.GaussianSpectrum(mean=w.GREEN, std=(5*w.MICRO))
# s = w.DeltaSpectrum(wavelength=w.GREEN)
z = np.linspace(-6*w.MICRO, 6*w.MICRO, 1000)
s.delay(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
s.plot(ax=ax1, wavelength=True)
s.plot_amplitude(z, ax=ax2)
plt.show()

### Compare different materials

#### Analitical vs constant vs matrix theory

In [ ]:
air = w.EmptySpace(z=z, name="analytic")
common_n = 1
fixed_dielectric = w.ConstantMaterial(z=z, n0=common_n, name="phase shift")
dielectric = w.SimpleDielectric(z=z, n0=1, name="transfer matrices")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

air.plot(ax1)
fixed_dielectric.plot(ax2)
dielectric.plot(ax3)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(z, air.energy_distribution(s,s), label="analitical")
ax1.plot(dielectric.z, dielectric.energy_distribution(s, s), label="numerical - wave transfer matrices")
ax1.plot(fixed_dielectric.z, fixed_dielectric.energy_distribution(s, s), label="numerical - phase shift")
ax1.legend()

air.transmit(s).plot(ax2)
dielectric.transmit(s).plot(ax2)
fixed_dielectric.transmit(s).plot(ax2)
plt.show()

### Layered material

In [ ]:
z_layered = np.linspace(-10*w.MICRO, 10*w.MICRO, 1000)
layered = w.LayeredMaterial(z=z_layered, n_layers=2, layer_width=2*w.MICRO, n0=1, n1=1.1)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

layered.shade_plot(ax1, alpha=0.2)
layered.plot(ax1, plot_imaginary=False)
layered.shade_plot(ax2, alpha=0.2)
layered.record(s, s)
ax2.plot(layered.z, layered.recent_energy, label="intensity")
ax2.legend()
layered.shade_plot(ax3, alpha=0.2)
ax3.plot(layered.z, layered.deposited_energy, label="cropped intensity")
ax3.legend()
plt.show()

### An example recording

In [ ]:
z = np.linspace(-6*w.MICRO, 6*w.MICRO, 1000)
dielectric = w.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
s0 = w.GaussianSpectrum(mean=w.GREEN, std=(5*w.MICRO))
dielectric.record(s0, s0)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(dielectric.z[:-1], dielectric.recent_energy[:-1])
ax1.set_title("interference")
dielectric.plot(plot_imaginary=False, ax=ax2)
plt.show()

In [ ]:
steps = 20
plt.rcParams['axes.prop_cycle'] = cycler('color', plt.get_cmap('viridis')(np.linspace(0, 1, steps+1)))

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

for pulse in range(steps):
    phase = 0
    s2 = s0 * np.exp(1j*phase)
    dielectric.plot(ax=ax2, plot_imaginary=False, alpha=0.5)
    dielectric.record(s0, s2)
    ax1.plot(dielectric.z[:-1], dielectric.recent_energy[:-1], label="pulse {}".format(pulse + 1), alpha=0.5)
    
plt.legend(loc=3)
plt.show()